<a href="https://colab.research.google.com/github/emredeveloper/Transformers--General-AI/blob/main/SLM_%2B_COT_FINETUNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
class ConversationDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=512):
        self.examples = []

        print("Processing conversations...")
        for item in tqdm(dataset):
            # Format conversation
            conversation = ""
            for turn in item['chosen']:
                role = turn['role']
                content = turn['content']
                if role == 'user':
                    conversation += f"Human: {content}\n"
                else:
                    conversation += f"Assistant: {content}\n"

            # Tokenize
            encodings = tokenizer(
                conversation,
                truncation=True,
                max_length=max_length,
                padding="max_length",
                return_tensors="pt"
            )

            self.examples.append({
                "input_ids": encodings["input_ids"][0],
                "attention_mask": encodings["attention_mask"][0],
                "labels": encodings["input_ids"][0].clone()
            })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

In [ ]:
def main():
    # Load dataset
    print("Loading dataset...")
    dataset = load_dataset("kenhktsui/longtalk-cot-v0.1")

    # Kullan küçük bir subset (test için)
    dataset['train'] = dataset['train'].select(range(1000))

    # Load model and tokenizer
    print("Loading model and tokenizer...")
    model_name = "HuggingFaceTB/SmolLM-135M"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.pad_token_id

    # Split dataset
    print("Splitting dataset...")
    train_size = int(0.9 * len(dataset['train']))
    val_size = len(dataset['train']) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset['train'],
        [train_size, val_size]
    )

    # Prepare datasets
    print("Preparing training dataset...")
    train_dataset = ConversationDataset(train_dataset, tokenizer)
    print("Preparing validation dataset...")
    eval_dataset = ConversationDataset(val_dataset, tokenizer)

    # Training arguments
    # Training arguments
    training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    fp16=True,
    report_to="none"
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    )

    # Start training
    trainer.train()

    # Save model
    model_save_path = "./fine_tuned_smolLM"
    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
# Model değerlendirme için gerekli fonksiyonlar ve test kodu
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def model_yukle():
    model_path = "./fine_tuned_smolLM"
    model = AutoModelForCausalLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

def yanit_uret(prompt, model, tokenizer, max_length=250):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    # CoT formatında prompt hazırla
    formatted_prompt = f"Question: {prompt}\nLet's solve this step by step:\n"

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Chain-of-Thought tarzında test soruları
test_ornekleri = [
    "What is the sum of the first 5 prime numbers?",
    "What is 1 + 1?"
]

# Modeli yükle ve test et
print("Model Evaluation Results:")
print("-" * 70)

model, tokenizer = model_yukle()

for ornek in test_ornekleri:
    print(f"\nQuestion: {ornek}")
    yanit = yanit_uret(ornek, model, tokenizer)
    print(f"Response:\n{yanit}")
    print("-" * 70)

In [ ]:
dataset = load_dataset("kenhktsui/longtalk-cot-v0.1")
dataset["train"][0]

In [ ]:
dataset["train"].shape